In [1]:

from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from boruta import BorutaPy 

    

In [2]:
def sensitivity_score(ypred,y):
    tp=np.sum((ypred==1) & (y==1))
   # print("tp")
    totmal=np.sum(y==1)
   # print(tp)
    #print("sum ypred:")
    #print(sum(ypred==1))
    #print("sum y:")
    #print(sum(y==1))
    #print("totmal")
    
    #Non mi ritrovo nel dividere tutto per "ypred.shape[0]" la sensibilità è TruePos/totmalati
    
    #print(totpos)
    print(tp/totmal)
    
    

In [3]:
#funzione per calcolare F_score (o almeno dovrebbe) TruePos/(TruePos+ 1/2 (FalsePos+FalseNeg)) 
def f_score(ypred, y):
    tp=np.sum((ypred==1) & (y==1))
    fp=np.sum(ypred==1)-tp
    tn=np.sum((ypred==0) & (y==0))
    fn=np.sum(ypred==0)-tn
    
    print(tp/(tp+0.5*(fp+fn)))

In [4]:

#Funzione per calcolare la specificità (o almeno dovrebbe) TrueNeg/TotSani

def specificity_score(ypred,y):
    tn=np.sum((ypred==0) & (y==0))
    totsani=np.sum(y==0)
    
    print (tn/totsani)

In [5]:
#carico il dataset
data = pd.read_csv(r'C:\Users\Utente\anaconda3\Lib\site-packages\pandas\io\data_covnet_score-imputed_missRF_increasing_1.txt')
print(data.shape)
nit=15

#il comando sotto non mi serve: Converte i dati categorici in dati classificabili.
#data = pd.get_dummies(data, drop_first=True, dummy_na=True)
#print(data.shape)

y=data['LABEL']
y=y.to_numpy()

parzial_features = list()

altri=data
altri=altri.to_numpy()


features = [f for f in data.columns if f not in ['LABEL']]
#print(len(features))

X = data[features].values

#ravel() mi fa diventare l'array monodimensionale.
Y = data['LABEL'].values.ravel()

rf = RandomForestClassifier(n_estimators = 100, max_depth=11, max_features=None, max_samples=0.75,class_weight="balanced")

boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=4141, max_iter = 50, perc = 40)



#creo un array monodimensionale lungo quanto "altri" ma solo di 0. 
y_pred=y-y


#crea un oggetto pronto ad operare: quando gli arriva in input qualcosa lo divide in 10 pezzettini con la stessa proporzione
skf = StratifiedKFold(n_splits=10)


for niter in range(nit):
    for train_index, test_index in skf.split(X, Y):

            X_train, X_test = X[train_index,:], X[test_index,:]
            #usa boruta per selezionare le features più disciriminative dal training set
            #prendi solo le features più significative sia nel training set che nel test set
            boruta_feature_selector.fit(X, Y)

            y_train, y_test = Y[train_index], Y[test_index]

            clf = rf.fit(X_train, y_train)
            y_pred[test_index] = clf.predict(X_test)

           # array = boruta_feature_selector.transform(X)

            indexes = np.where(boruta_feature_selector.support_ == True)
            for x in np.nditer(indexes):
                parzial_features.append(features[x])

        #fine ciclo for
        
    #calcolare media di sensitivity ecc. QUI
  
    
#X_filtered = boruta_feature_selector.transform(X)
#final_features = list()
#indexes = np.where(boruta_feature_selector.support_ == True)
#for x in np.nditer(indexes):
 #   final_features.append(features[x])
#print(final_features)
print()
for i in features:
  c=parzial_features.count(i)
  print(i,'        ', c)



(301, 42)
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	19
Tentative: 	13
Rejected: 	9
Iteration: 	9 / 50
Confirmed: 	19
Tentative: 	13
Rejected: 	9
Iteration: 	10 / 50
Confirmed: 	19
Tentative: 	13
Rejected: 	9
Iteration: 	11 / 50
Confirmed: 	19
Tentative: 	13
Rejected: 	9
Iteration: 	12 / 50
Confirmed: 	19
Tentative: 	10
Rejected: 	12
Iteration: 	13 / 50
Confirmed: 	19
Tentative: 	10
Rejected: 	12
Iteration: 	14 / 50
Confirmed: 	19
Tentative: 	10
Rejected: 	12
Iteration: 	15 / 50
Confirmed: 	19
Tentative: 	10
Rejected: 	12
Iteration: 	16 / 50
Confirmed: 	19
Tentative: 	10
Rejecte

Iteration: 	31 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	32 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	33 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	34 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	35 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	36 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	37 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	38 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	39 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	40 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	41 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	42 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	43 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	44 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	45 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	46 / 50
Confirmed: 	19
Tentative: 	0
Reject

Iteration: 	17 / 50
Confirmed: 	21
Tentative: 	8
Rejected: 	12
Iteration: 	18 / 50
Confirmed: 	21
Tentative: 	8
Rejected: 	12
Iteration: 	19 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	20 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	21 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	22 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	23 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	24 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	25 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	26 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	27 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	28 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	29 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	30 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	31 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	32 / 50
Confirmed: 	21
Tentative: 	2
Reject

Iteration: 	9 / 50
Confirmed: 	21
Tentative: 	13
Rejected: 	7
Iteration: 	10 / 50
Confirmed: 	21
Tentative: 	13
Rejected: 	7
Iteration: 	11 / 50
Confirmed: 	21
Tentative: 	13
Rejected: 	7
Iteration: 	12 / 50
Confirmed: 	21
Tentative: 	6
Rejected: 	14
Iteration: 	13 / 50
Confirmed: 	21
Tentative: 	6
Rejected: 	14
Iteration: 	14 / 50
Confirmed: 	21
Tentative: 	6
Rejected: 	14
Iteration: 	15 / 50
Confirmed: 	21
Tentative: 	6
Rejected: 	14
Iteration: 	16 / 50
Confirmed: 	21
Tentative: 	6
Rejected: 	14
Iteration: 	17 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	18 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	19 / 50
Confirmed: 	21
Tentative: 	5
Rejected: 	15
Iteration: 	20 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	21 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	22 / 50
Confirmed: 	21
Tentative: 	4
Rejected: 	16
Iteration: 	23 / 50
Confirmed: 	21
Tentative: 	3
Rejected: 	17
Iteration: 	24 / 50
Confirmed: 	21
Tentative: 	3
Rejecte

Iteration: 	46 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	47 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	48 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	49 / 50
Confirmed: 	19
Tentative: 	0
Rejected: 	22


BorutaPy finished running.

Iteration: 	50 / 50
Confirmed: 	19
Tentative: 	0
Rejected: 	22
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	22
Tentative: 	12
Rejected: 	7
Iteration: 	9 / 50
Confirmed: 	22
Tentative: 	12
Rejected: 	7
Iteration: 	10 / 50
Confirmed: 	22
Tentative: 	12
Rejected: 	7
Iteration: 	11 / 50
Confirmed: 	22
Tentat

...

Iteration: 	12 / 50
Confirmed: 	20
Tentative: 	11
Rejected: 	10
Iteration: 	13 / 50
Confirmed: 	20
Tentative: 	11
Rejected: 	10
Iteration: 	14 / 50
Confirmed: 	20
Tentative: 	10
Rejected: 	11
Iteration: 	15 / 50
Confirmed: 	20
Tentative: 	10
Rejected: 	11
Iteration: 	16 / 50
Confirmed: 	20
Tentative: 	8
Rejected: 	13
Iteration: 	17 / 50
Confirmed: 	20
Tentative: 	8
Rejected: 	13
Iteration: 	18 / 50
Confirmed: 	20
Tentative: 	8
Rejected: 	13
Iteration: 	19 / 50
Confirmed: 	20
Tentative: 	8
Rejected: 	13
Iteration: 	20 / 50
Confirmed: 	20
Tentative: 	7
Rejected: 	14
Iteration: 	21 / 50
Confirmed: 	20
Tentative: 	7
Rejected: 	14
Iteration: 	22 / 50
Confirmed: 	20
Tentative: 	5
Rejected: 	16
Iteration: 	23 / 50
Confirmed: 	20
Tentative: 	5
Rejected: 	16
Iteration: 	24 / 50
Confirmed: 	20
Tentative: 	5
Rejected: 	16
Iteration: 	25 / 50
Confirmed: 	20
Tentative: 	5
Rejected: 	16
Iteration: 	26 / 50
Confirmed: 	20
Tentative: 	4
Rejected: 	17
Iteration: 	27 / 50
Confirmed: 	20
Tentative: 	4
Re

Iteration: 	42 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	43 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	44 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	45 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	46 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	47 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	48 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	49 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20


BorutaPy finished running.

Iteration: 	50 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tent

Iteration: 	31 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	32 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	33 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	34 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	35 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	36 / 50
Confirmed: 	19
Tentative: 	3
Rejected: 	19
Iteration: 	37 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	38 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	39 / 50
Confirmed: 	19
Tentative: 	2
Rejected: 	20
Iteration: 	40 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	41 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	42 / 50
Confirmed: 	19
Tentative: 	1
Rejected: 	21
Iteration: 	43 / 50
Confirmed: 	19
Tentative: 	0
Rejected: 	22


BorutaPy finished running.

Iteration: 	44 / 50
Confirmed: 	19
Tentative: 	0
Rejected: 	22
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirme

Iteration: 	34 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	35 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	36 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	37 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	38 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	39 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	40 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20


BorutaPy finished running.

Iteration: 	41 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	20
Tenta

Iteration: 	32 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	33 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	34 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	35 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	36 / 50
Confirmed: 	21
Tentative: 	1
Rejected: 	19
Iteration: 	37 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20


BorutaPy finished running.

Iteration: 	38 / 50
Confirmed: 	21
Tentative: 	0
Rejected: 	20
Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	41
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	20
Tentative: 	12
Rejected: 	9
Iteration: 	9 / 50
Confirmed: 	20
Tentat